# Single Trajectory with Feedback

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qutip as qt
import cmath
import statsmodels.api as sm
import scipy as sp
%run Functions.ipynb

### State vectors and field operators

Definition of the states corresponding to the two potential wells:

In [ ]:
psi_0 = qt.fock(2,0)
psi_1 = qt.fock(2,1)
O_I = qt.tensor(psi_0,psi_1)
I_O = qt.tensor(psi_1,psi_0)

Initial conditions:

In [ ]:
initial_ket = +np.cos(np.pi/8)*I_O+1j*np.sin(np.pi/8)*O_I
initial_ket.dag()*initial_ket

Definition of the field operators:

In [ ]:
a_1 = qt.tensor(qt.destroy(2), qt.qeye(2))
a_2 = qt.tensor(qt.qeye(2), qt.destroy(2))

Definition of the number operators:

In [ ]:
N_1 = a_1.dag()*a_1
N_2 = a_2.dag()*a_2

Bloch operators:

In [ ]:
x = a_1.dag()*a_2+a_2.dag()*a_1
y = 1j*(-a_1.dag()*a_2+a_2.dag()*a_1)
z = a_1.dag()*a_1-a_2.dag()*a_2

Expectation values initial conditions:

In [ ]:
print('expected initial x', qt.expect(x, initial_ket))
print('expected initial y', qt.expect(y, initial_ket))
print('expected initial z', qt.expect(z, initial_ket))

Initial density operator:

In [ ]:
rho_0 = initial_ket*initial_ket.dag()

In [ ]:
rho_0

### Feedback Master Equation

Parameters of the feedback ME:

In [ ]:
omega_1 = 0 
omega_2 = 0
Omega = 0.3 #Exchange energy
mu = 0.02 #Feedback strenght
T = 10. #Tunneling coefficient
chi = 0.1 #Tunneling coefficient variation
gamma = abs(chi)/abs(T)
dt = 0.1
times = np.arange(0, 1000000, dt)
NUMBER_OF_TRAJECTORIES = 1

Operators:

In [ ]:
F = mu*x #feedback operator 
A = T + chi*N_1 
Jump =A-1j*F#Jump operator for the ME in Lindblad form
H = Omega*(a_1.dag()*a_2 + a_2.dag()*a_1)+1/2*(A*F+F*A)-T*F #Hamiltonian

In [ ]:
#noise order
print('noise order', 1/np.sqrt(dt))

Current operators:

In [ ]:
I = abs(T)*(1 + 2*gamma*np.cos(cmath.phase(T) - cmath.phase(chi))*(a_1.dag()*a_1 - a_2.dag()*a_2))
I_1 = chi*z

In [ ]:
I_1

### Simulation

In [ ]:
stoc_solution = qt.smesolve(H, rho_0, times,
                            c_ops=[],
                            sc_ops=[Jump],
                            m_ops = [I_1],
                            ntraj=NUMBER_OF_TRAJECTORIES,
                            nsubsteps=100,
                            store_measurement=True,
                            dW_factors=[1.])

### Single Trajectory solution

In [ ]:
fig, ax = plt.subplots()
ax.set_title('Stochastic Master Equation - double dot, single trajectory')
ax.plot(times,stoc_solution.measurement[0],
        'r', lw=2, label=r'$J_x$')

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

In [ ]:
np.savetxt('SingleFeed/single_feed.txt', stoc_solution.measurement[0])